# Pre-Processing of Data

### Pre-Processing of News_Excerpts


In [ ]:
import os
import pandas as pd
import re
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import concurrent.futures

# Perform entity extraction using spaCy
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")  # Load spaCy model
    doc = nlp(text)
    entities = {entity.label_: [] for entity in doc.ents}
    for entity in doc.ents:
        entities[entity.label_].append(entity.text)
    return entities

# Calculate subjectivity and polarity with TextBlob
def get_subjectivity_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.subjectivity, blob.sentiment.polarity

# Remove redundancy based on cosine similarity of TF-IDF vectors
def remove_redundancy(texts):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    unique_texts = []
    seen = set()
    for idx, row in enumerate(similarity_matrix):
        if idx not in seen:
            unique_texts.append(texts[idx])
            seen.update({jdx for jdx, sim in enumerate(row) if sim >= 0.8 and idx != jdx})  # 0.8 threshold
    return unique_texts

# Preprocess and perform all tasks
def preprocess_data(file_path):
    df = load_data(file_path)
    cleaned_texts = []
    all_entities = []
    all_sentiments = []
    subjectivities = []
    polarities = []

    # Process data in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        
        for idx, text in enumerate(df['Text']):
            futures.append(executor.submit(process_text, idx, text, cleaned_texts, all_entities, all_sentiments, subjectivities, polarities))
        
        # Wait for all futures to complete
        concurrent.futures.wait(futures)
    
    # Redundancy removal
    unique_texts = remove_redundancy(cleaned_texts)

    # Compile results
    df['Cleaned Text'] = cleaned_texts
    df['Entities'] = all_entities
    df['Sentiment'] = all_sentiments
    df['Subjectivity'] = subjectivities
    df['Polarity'] = polarities

    # Save processed data to a new CSV
    df.to_csv('processed_data_no_api.csv', index=False)
    print("Processing completed and saved to 'processed_data_no_api.csv'")
    return df

# Function to process each text (called by ThreadPoolExecutor)
def process_text(idx, text, cleaned_texts, all_entities, all_sentiments, subjectivities, polarities):
    try:
        # Clean the text
        cleaned_text = clean_text(text)
        cleaned_texts.append(cleaned_text)

        # Extract entities
        entities = extract_entities(cleaned_text)
        all_entities.append(entities)

        # Sentiment analysis
        blob = TextBlob(cleaned_text)
        sentiment = "Positive" if blob.sentiment.polarity > 0 else "Negative" if blob.sentiment.polarity < 0 else "Neutral"
        all_sentiments.append(sentiment)

        # Get subjectivity and polarity
        subjectivity, polarity = get_subjectivity_polarity(cleaned_text)
        subjectivities.append(subjectivity)
        polarities.append(polarity)

        # Print progress
        print(f"Processed text {idx + 1} - Sentiment: {sentiment}")
    except Exception as e:
        print(f"Error processing text {idx + 1}: {e}")
        raise  # Raise the exception to stop the execution

# Clean the text data (remove noise, special characters, and apply lowercasing)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    return text

# Load the CSV data
def load_data(file_path):
    return pd.read_csv(file_path)

if __name__ == "__main__":
    file_path = "news_excerpts_parsed.csv"  # Path to your CSV file
    try:
        processed_data = preprocess_data(file_path)
        print(processed_data.head())
    except Exception as e:
        print(f"An error occurred during processing: {e}")


## Pre-Processing for Wiki_leaks

In [1]:
import os
import pandas as pd
import re
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import concurrent.futures

# Perform entity extraction using spaCy
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")  # Load spaCy model
    doc = nlp(text)
    entities = {entity.label_: [] for entity in doc.ents}
    for entity in doc.ents:
        entities[entity.label_].append(entity.text)
    return entities

# Calculate subjectivity and polarity with TextBlob
def get_subjectivity_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.subjectivity, blob.sentiment.polarity

# Remove redundancy based on cosine similarity of TF-IDF vectors
def remove_redundancy(texts):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    unique_texts = []
    seen = set()
    for idx, row in enumerate(similarity_matrix):
        if idx not in seen:
            unique_texts.append(texts[idx])
            seen.update({jdx for jdx, sim in enumerate(row) if sim >= 0.8 and idx != jdx})  # 0.8 threshold
    return unique_texts

# Preprocess and perform all tasks
def preprocess_data(file_path):
    df = load_data(file_path)
    cleaned_texts = []
    all_entities = []
    all_sentiments = []
    subjectivities = []
    polarities = []

    # Process data in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        
        for idx, text in enumerate(df['Text']):
            futures.append(executor.submit(process_text, idx, text, cleaned_texts, all_entities, all_sentiments, subjectivities, polarities))
        
        # Wait for all futures to complete
        concurrent.futures.wait(futures)
    
    # Redundancy removal
    unique_texts = remove_redundancy(cleaned_texts)

    # Compile results
    df['Cleaned Text'] = cleaned_texts
    df['Entities'] = all_entities
    df['Sentiment'] = all_sentiments
    df['Subjectivity'] = subjectivities
    df['Polarity'] = polarities

    # Save processed data to a new CSV
    df.to_csv('processed_data_wikileaks.csv', index=False)
    print("Processing completed and saved to 'processed_data_no_api.csv'")
    return df

# Function to process each text (called by ThreadPoolExecutor)
def process_text(idx, text, cleaned_texts, all_entities, all_sentiments, subjectivities, polarities):
    try:
        # Clean the text
        cleaned_text = clean_text(text)
        cleaned_texts.append(cleaned_text)

        # Extract entities
        entities = extract_entities(cleaned_text)
        all_entities.append(entities)

        # Sentiment analysis
        blob = TextBlob(cleaned_text)
        sentiment = "Positive" if blob.sentiment.polarity > 0 else "Negative" if blob.sentiment.polarity < 0 else "Neutral"
        all_sentiments.append(sentiment)

        # Get subjectivity and polarity
        subjectivity, polarity = get_subjectivity_polarity(cleaned_text)
        subjectivities.append(subjectivity)
        polarities.append(polarity)

        # Print progress
        print(f"Processed text {idx + 1} - Sentiment: {sentiment}")
    except Exception as e:
        print(f"Error processing text {idx + 1}: {e}")
        raise  # Raise the exception to stop the execution

# Clean the text data (remove noise, special characters, and apply lowercasing)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    return text

# Load the CSV data
def load_data(file_path):
    return pd.read_csv(file_path)

if __name__ == "__main__":
    file_path = "wikileaks_parsed.csv"  # Path to your CSV file
    try:
        processed_data = preprocess_data(file_path)
        print(processed_data.head())
    except Exception as e:
        print(f"An error occurred during processing: {e}")


Processed text 3 - Sentiment: Positive
Processed text 4 - Sentiment: Positive
Processed text 1 - Sentiment: Negative
Processed text 2 - Sentiment: Positive
Processed text 5 - Sentiment: Negative
Processed text 6 - Sentiment: Positive
Processed text 7 - Sentiment: Negative
Processed text 8 - Sentiment: Neutral
Processed text 9 - Sentiment: Negative
Processed text 10 - Sentiment: Positive
Processed text 12 - Sentiment: Positive
Processed text 11 - Sentiment: Positive
Processed text 13 - Sentiment: Negative
Processed text 14 - Sentiment: Negative
Processed text 15 - Sentiment: Negative
Processed text 16 - Sentiment: Positive
Processed text 17 - Sentiment: Positive
Processed text 18 - Sentiment: Negative
Processed text 19 - Sentiment: Negative
Processed text 20 - Sentiment: Negative
Processed text 21 - Sentiment: Positive
Processed text 22 - Sentiment: Negative
Processed text 23 - Sentiment: Negative
Processed text 26 - Sentiment: Positive
Processed text 25 - Sentiment: Positive
Processed 

# Extracting the Relationships using LLMs

## Performed for cleaned news_excerpts

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI  # Correct import
from langchain.chains import LLMChain
from tqdm import tqdm
import json
import csv

# Load environment variables
load_dotenv()

# Set up Azure OpenAI credentials
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_APIKEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["OPENAI_API_DEPLOYMENT_NAME"] = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name=os.environ["OPENAI_API_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],  # Use the correct endpoint parameter
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.2,
    model=os.getenv("AZURE_OPENAI_MODEL_NAME")  # Automatically use the model from .env
)

# Define the prompt template
prompt_template = """
Extract as many entity-relationship-entity (E-R-E) triples as possible from the following text.

Each triple should follow the format:
(Entity1, Relationship, Entity2)
where:
- "Entity1" and "Entity2" are named entities.
- "Relationship" is the connection between them.

Return output in valid JSON format as a list of tuples:
[
    ("Entity1", "Relationship", "Entity2"),
    ("Entity3", "Relationship", "Entity4"),
    ...
]

Text: "{text}"
Output:
"""

# Create the PromptTemplate object
prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template
)

# Define the LLM chain
chain = LLMChain(llm=llm, prompt=prompt)

def extract_triples(cleaned_text):
    try:
        response = chain.run({"text": cleaned_text})
        response_cleaned = response.strip("`").strip().removeprefix("json").strip()  # Remove 'json' prefix and cleanup

        # Try parsing the cleaned response as JSON or evaluate tuples
        if response_cleaned.startswith("[") and response_cleaned.endswith("]"):
            try:
                extracted_data = json.loads(response_cleaned)  # For valid JSON
            except json.JSONDecodeError:
                extracted_data = eval(response_cleaned)  # For tuple-based response
            return extracted_data
        else:
            print(f"Warning: Unexpected format for response: {response_cleaned}")
            return []
    except Exception as e:
        print(f"Error processing text: {e}")
        return []

# Input CSV and output CSV paths
input_csv_path = "processed_data_no_api.csv"  # Input file path
output_csv_path = "entity_relationship_triples.csv"  # Output file path

# Load the input CSV
df = pd.read_csv(input_csv_path)

# Ensure the correct third column is used (assuming 'Cleaned Text' is in the 3rd column)
third_column_name = df.columns[2]
print(f"Using column: {third_column_name}")

# Prepare the output CSV with headers if it doesn't exist
if not os.path.exists(output_csv_path):
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Entity1", "Relationship", "Entity2", "SourceRow"])
        writer.writeheader()

# Process each row in the 'Cleaned Text' column and save to the output CSV
for index, cleaned_text in tqdm(enumerate(df[third_column_name]), desc="Processing rows"):
    extracted_triples = extract_triples(cleaned_text)
    with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Entity1", "Relationship", "Entity2", "SourceRow"])
        for triple in extracted_triples:
            if len(triple) == 3:
                writer.writerow({"Entity1": triple[0], "Relationship": triple[1], "Entity2": triple[2], "SourceRow": index})

print(f"Extraction complete. Results saved to {output_csv_path}.")


## Performed for cleaned wiki_leaks

In [14]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI  # Correct import
from langchain.chains import LLMChain
from tqdm import tqdm
import json
import csv

# Load environment variables
load_dotenv()

# Set up Azure OpenAI credentials
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_APIKEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["OPENAI_API_DEPLOYMENT_NAME"] = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name=os.environ["OPENAI_API_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],  # Use the correct endpoint parameter
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.2,
    model=os.getenv("AZURE_OPENAI_MODEL_NAME")  # Automatically use the model from .env
)

# Define the prompt template
prompt_template = """

Extract named entities and the relationships between them from the following text.

For each entity-relationship pair, return the entities and the relationship in this format:
(Entity1, Relationship, Entity2)
Where:
- "Entity1" and "Entity2" are the named entities.
- "Relationship" is the connection between the entities.

Additionally, include any relevant details such as dates, times, or conditions that describe the relationship.

Return the output as a list of dictionaries, each with the following keys:
- "Entity1": The first named entity
- "Relationship": The relationship between Entity1 and Entity2
- "Entity2": The second named entity
- "Attributes": A dictionary containing any relevant additional details (like dates, conditions, etc.)

Text: "{text}"
Output:
"""


# Create the PromptTemplate object
prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template
)

# Define the LLM chain
chain = LLMChain(llm=llm, prompt=prompt)

def extract_triples_with_attributes(cleaned_text):
    try:
        # Log cleaned text to ensure it's being passed correctly
        print(f"Sending text to model: {cleaned_text[:100]}...")  # Only print the first 100 chars for safety

        # Get the model's response
        response = chain.run({"text": cleaned_text})

        # Log the raw response to understand what we're receiving
        print(f"Raw response from model: {response}")

        # Ensure the response is valid
        if not response:
            print("Error: Received empty response from model.")
            return []

        # Clean the response and check the format
        response_cleaned = response.strip("`").strip().removeprefix("json").strip()

        # Log cleaned response to further inspect
        print(f"Cleaned response: {response_cleaned}")

        if response_cleaned.startswith("[") and response_cleaned.endswith("]"):
            try:
                # Attempt to parse the response as JSON
                extracted_data = json.loads(response_cleaned)
                return extracted_data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                print(f"Raw output could not be parsed as JSON: {response_cleaned}")
                return []
        else:
            print(f"Warning: Unexpected format for response: {response_cleaned}")
            return []

    except Exception as e:
        print(f"Error processing text: {e}")
        return []

# Input CSV and output CSV paths
input_csv_path = "wiki_cleaned.csv"  # Input file path
output_csv_path = "wiki_ERE_with_attributes.csv"  # Output file path

# Load the input CSV
df = pd.read_csv(input_csv_path)

# Ensure the correct third column is used (assuming 'Cleaned Text' is in the 3rd column)
third_column_name = df.columns[2]
print(f"Using column: {third_column_name}")

# Prepare the output CSV with headers if it doesn't exist
if not os.path.exists(output_csv_path):
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Entity1", "Relationship", "Entity2", "SourceRow", "Attributes"])
        writer.writeheader()

# Process each row in the 'Cleaned Text' column and save to the output CSV
for index, cleaned_text in tqdm(enumerate(df[third_column_name]), desc="Processing rows"):
    extracted_triples = extract_triples_with_attributes(cleaned_text)
    
    # If the triples are extracted, save them
    if extracted_triples:
        with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=["Entity1", "Relationship", "Entity2", "SourceRow", "Attributes"])
            for triple in extracted_triples:
                if len(triple) == 3:
                    writer.writerow({
                        "Entity1": triple["Entity1"],
                        "Relationship": triple["Relationship"],
                        "Entity2": triple["Entity2"],
                        "SourceRow": index,
                        "Attributes": json.dumps(triple["Attributes"])  # Store the attributes as a JSON string
                    })

print(f"Extraction complete. Results saved to {output_csv_path}.")


Using column: Cleaned Text


Processing rows: 0it [00:00, ?it/s]

Sending text to model: pristina airport  possible administrative irregularity regarding tender procedures involving vendor ...


Processing rows: 1it [00:06,  6.75s/it]

Raw response from model: ```json
[
    {
        "Entity1": "vendor 1",
        "Relationship": "has the same owner as",
        "Entity2": "vendor 2",
        "Attributes": {}
    },
    {
        "Entity1": "vendor 1",
        "Relationship": "submitted tenders for",
        "Entity2": "pristina airport",
        "Attributes": {
            "year": 2003,
            "number_of_times": 3
        }
    },
    {
        "Entity1": "vendor 2",
        "Relationship": "submitted tenders for",
        "Entity2": "pristina airport",
        "Attributes": {
            "year": 2003,
            "number_of_times": 3
        }
    },
    {
        "Entity1": "vendor 1",
        "Relationship": "won tender for",
        "Entity2": "supply and mounting of sonic system in the fire station building",
        "Attributes": {
            "year": 2003,
            "price": 1620
        }
    },
    {
        "Entity1": "vendor 2",
        "Relationship": "won tender for",
        "Entity2": "supply a

...

Raw response from model: ```json
[
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "constitutes",
        "Entity2": "State Owned Enterprise",
        "Attributes": {
            "Law": "Yugoslavian Law"
        }
    },
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "includes",
        "Entity2": "Assets",
        "Attributes": {
            "Details": [
                "Runways",
                "Terminal Buildings",
                "Hanger",
                "Fuel Storage Facilities",
                "Equipment"
            ]
        }
    },
    {
        "Entity1": "Public Enterprise Airport Pristina (PEAP)",
        "Relationship": "maintained by",
        "Entity2": "Military Units of the Kosovo Force (KFOR)",
        "Attributes": {
            "Period": "2001 to 2003"
        }
    },
    {
        "Entity1": "United Nations Security Council Resolution 1244",
        "Relationship": "dated",
        "Entity2

Processing rows: 142it [10:50,  5.45s/it]

Raw response from model: ```json
[
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "constitutes",
        "Entity2": "state owned enterprise",
        "Attributes": {
            "Law": "Yugoslavian law"
        }
    },
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "includes",
        "Entity2": "runways, terminal buildings, hanger, fuel storage facilities, and equipment",
        "Attributes": {}
    },
    {
        "Entity1": "Public Enterprise Airport Pristina (PEAP)",
        "Relationship": "maintained by",
        "Entity2": "military units of the Kosovo Force (KFOR)",
        "Attributes": {
            "Period": "2001 until 2003"
        }
    },
    {
        "Entity1": "United Nations Security Council Resolution 1244",
        "Relationship": "dated",
        "Entity2": "10 June 1999",
        "Attributes": {}
    },
    {
        "Entity1": "UNMIK Regulation No 11999",
        "Relationship": "dated",


Processing rows: 143it [10:54,  4.57s/it]

Raw response from model: ```json
[
    {
        "Entity1": "Jean-David Levitte",
        "Relationship": "reveals frustration towards",
        "Entity2": "Iraq Survey Group",
        "Attributes": {
            "Details": "Frustration regarding a report's findings"
        }
    },
    {
        "Entity1": "Iraq Survey Group",
        "Relationship": "found that",
        "Entity2": "UN Oil-for-Food Program",
        "Attributes": {
            "Details": "Possibly misused by French companies"
        }
    },
    {
        "Entity1": "Jean-David Levitte",
        "Relationship": "states that it is scandalous that",
        "Entity2": "US Companies",
        "Attributes": {
            "Details": "No US companies are named in the report"
        }
    },
    {
        "Entity1": "Jean-David Levitte",
        "Relationship": "claims that many",
        "Entity2": "French Companies",
        "Attributes": {
            "Details": "Named in the report were actually subsidiaries of US fi

In [15]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI  # Correct import
from langchain.chains import LLMChain
from tqdm import tqdm
import json
import csv

# Load environment variables
load_dotenv()

# Set up Azure OpenAI credentials
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_APIKEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["OPENAI_API_DEPLOYMENT_NAME"] = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name=os.environ["OPENAI_API_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],  # Use the correct endpoint parameter
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.2,
    model=os.getenv("AZURE_OPENAI_MODEL_NAME")  # Automatically use the model from .env
)

# Define the prompt template
prompt_template = """

Extract named entities and the relationships between them from the following text.

For each entity-relationship pair, return the entities and the relationship in this format:
(Entity1, Relationship, Entity2)
Where:
- "Entity1" and "Entity2" are the named entities.
- "Relationship" is the connection between the entities.

Additionally, include any relevant details such as dates, times, or conditions that describe the relationship.

Return the output as a list of dictionaries, each with the following keys:
- "Entity1": The first named entity
- "Relationship": The relationship between Entity1 and Entity2
- "Entity2": The second named entity
- "Attributes": A dictionary containing any relevant additional details (like dates, conditions, etc.)

Text: "{text}"
Output:
"""


# Create the PromptTemplate object
prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template
)

# Define the LLM chain
chain = LLMChain(llm=llm, prompt=prompt)

def extract_triples_with_attributes(cleaned_text):
    try:
        # Log cleaned text to ensure it's being passed correctly
        print(f"Sending text to model: {cleaned_text[:100]}...")  # Only print the first 100 chars for safety

        # Get the model's response
        response = chain.run({"text": cleaned_text})

        # Log the raw response to understand what we're receiving
        print(f"Raw response from model: {response}")

        # Ensure the response is valid
        if not response:
            print("Error: Received empty response from model.")
            return []

        # Clean the response and check the format
        response_cleaned = response.strip("`").strip().removeprefix("json").strip()

        # Log cleaned response to further inspect
        print(f"Cleaned response: {response_cleaned}")

        if response_cleaned.startswith("[") and response_cleaned.endswith("]"):
            try:
                # Attempt to parse the response as JSON
                extracted_data = json.loads(response_cleaned)
                return extracted_data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                print(f"Raw output could not be parsed as JSON: {response_cleaned}")
                return []
        else:
            print(f"Warning: Unexpected format for response: {response_cleaned}")
            return []

    except Exception as e:
        print(f"Error processing text: {e}")
        return []

# Input CSV and output CSV paths
input_csv_path = "wiki_cleaned.csv"  # Input file path
output_csv_path = "wiki_ERE_attributes.csv"  # Output file path

# Load the input CSV
df = pd.read_csv(input_csv_path)

# Ensure the correct third column is used (assuming 'Cleaned Text' is in the 3rd column)
third_column_name = df.columns[2]
print(f"Using column: {third_column_name}")

# Prepare the output CSV with headers if it doesn't exist
if not os.path.exists(output_csv_path):
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Entity1", "Relationship", "Entity2", "SourceRow", "Attributes"])
        writer.writeheader()

# Process each row in the 'Cleaned Text' column and save to the output CSV
for index, cleaned_text in tqdm(enumerate(df[third_column_name]), desc="Processing rows"):
    extracted_triples = extract_triples_with_attributes(cleaned_text)
    
    # If the triples are extracted, save them
    if extracted_triples:
        with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=["Entity1", "Relationship", "Entity2", "SourceRow", "Attributes"])
            for triple in extracted_triples:
                if len(triple) == 3:
                    writer.writerow({
                        "Entity1": triple["Entity1"],
                        "Relationship": triple["Relationship"],
                        "Entity2": triple["Entity2"],
                        "SourceRow": index,
                        "Attributes": json.dumps(triple["Attributes"])  # Store the attributes as a JSON string
                    })

print(f"Extraction complete. Results saved to {output_csv_path}.")


Using column: Cleaned Text


Processing rows: 0it [00:00, ?it/s]

Sending text to model: pristina airport  possible administrative irregularity regarding tender procedures involving vendor ...


Processing rows: 1it [00:06,  6.71s/it]

Raw response from model: ```json
[
    {
        "Entity1": "vendor 1",
        "Relationship": "has owner",
        "Entity2": "Kosovo citizen",
        "Attributes": {}
    },
    {
        "Entity1": "vendor 2",
        "Relationship": "has owner",
        "Entity2": "Kosovo citizen",
        "Attributes": {}
    },
    {
        "Entity1": "vendor 1",
        "Relationship": "is a shareholder of",
        "Entity2": "vendor 2",
        "Attributes": {
            "Shareholding": "51%"
        }
    },
    {
        "Entity1": "vendor 1",
        "Relationship": "submitted tenders",
        "Entity2": "Pristina airport",
        "Attributes": {
            "Year": 2003,
            "Number of submissions": 3
        }
    },
    {
        "Entity1": "vendor 2",
        "Relationship": "submitted tenders",
        "Entity2": "Pristina airport",
        "Attributes": {
            "Year": 2003,
            "Number of submissions": 3
        }
    },
    {
        "Entity1": "vendor 1"

...

Raw response from model: ```json
[
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "constitutes",
        "Entity2": "state owned enterprise",
        "Attributes": {
            "Law": "Yugoslavian law"
        }
    },
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "includes",
        "Entity2": "runways, terminal buildings, hanger, fuel storage facilities and equipment",
        "Attributes": {}
    },
    {
        "Entity1": "Public Enterprise Airport Pristina (PEAP)",
        "Relationship": "maintained by",
        "Entity2": "military units of the Kosovo Force (KFOR)",
        "Attributes": {
            "Period": "2001 until 2003"
        }
    },
    {
        "Entity1": "United Nations Security Council Resolution 1244",
        "Relationship": "dated",
        "Entity2": "10 June 1999",
        "Attributes": {}
    },
    {
        "Entity1": "UNMIK Regulation No 11999",
        "Relationship": "dated",
 

Processing rows: 142it [11:23,  3.84s/it]

Raw response from model: ```json
[
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "constitutes",
        "Entity2": "state owned enterprise",
        "Attributes": {
            "Law": "Yugoslavian law"
        }
    },
    {
        "Entity1": "Pristina International Airport",
        "Relationship": "includes",
        "Entity2": "runways, terminal buildings, hanger, fuel storage facilities, and equipment",
        "Attributes": {}
    },
    {
        "Entity1": "Public Enterprise Airport Pristina (PEAP)",
        "Relationship": "maintained by",
        "Entity2": "military units of the Kosovo Force (KFOR)",
        "Attributes": {
            "Period": "2001 until 2003"
        }
    },
    {
        "Entity1": "United Nations Security Council Resolution 1244",
        "Relationship": "dated",
        "Entity2": "10 June 1999",
        "Attributes": {}
    },
    {
        "Entity1": "UNMIK Regulation No 11999",
        "Relationship": "dated",


Processing rows: 143it [11:27,  4.81s/it]

Raw response from model: ```json
[
    {
        "Entity1": "Jean-David Levitte",
        "Relationship": "reveals frustration towards",
        "Entity2": "Iraq Survey Group",
        "Attributes": {
            "Details": "Frustration regarding a report's findings"
        }
    },
    {
        "Entity1": "Iraq Survey Group",
        "Relationship": "finds possible misuse of",
        "Entity2": "UN Oil-for-Food Program",
        "Attributes": {
            "Details": "Report indicates misuse by French companies"
        }
    },
    {
        "Entity1": "Jean-David Levitte",
        "Relationship": "claims that",
        "Entity2": "US companies",
        "Attributes": {
            "Details": "No US companies are named in the report"
        }
    },
    {
        "Entity1": "Jean-David Levitte",
        "Relationship": "claims that many",
        "Entity2": "French companies",
        "Attributes": {
            "Details": "Named in the report are subsidiaries of US firms"
      